<a href="https://colab.research.google.com/github/RaianaSeixas/Portifolio/blob/main/Optimization/L_SHADE_optimizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import scipy.stats
import random

In [ ]:
cols=10
dim = cols
NPAR1 = 100
ITERATIONS = 1000
MAX=np.repeat(100,dim)
MIN=np.repeat(-100,dim)
popsize = NPAR1
H = 6
kH = 0 # kH is used to work with H  
terminal = 0.1
p=0.1
SF=[];SCR=[];MF=[];MCR=[];Xarq=[]
MFi=0.5
MCRi=0.5

In [ ]:
# HappyCat function
# global mimima 0 at (-1,-1,...,-1)
# Not convex and non separable it is usually evaluated at xi =[(-600,600),..(-600,600)]

def fobj(x): 
    alpha=1. / 8
    s = sum(x**2) 
    return ((s - len(x))**2)**alpha + (s / 2 + sum(x)) / len(x) + 0.5 

In [ ]:
def LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fitness,X,Xarq,FES):

  SF,SCR,MF,MCR,p, terminal, Narquive,H,kH = setTUNE

  fmutant = []
  Xmutant = []
  fx=[]; fu=[] # to select the wij Lehmer Mean.
  # setTUNE = [SF,SCR,MF,MCR,p, terminal, Narquive,H] 
  dim = X.shape[1]
  #best_number = int(p*popsize)
  if (p*popsize<1):
    best_number=1
  else:
    best_number = int(p*popsize)
  justGoOUT=1

  if(justGoOUT==1):
    SCR =[];SF=[];
    for i in range(popsize):
      ri = random.randint(0,H-1) # line 7
      miF = MF[ri]
      miCR = MCR[ri]
      if(MCR[ri] == terminal):
        miCR = 0
        crossp = 0
      else:
        crossp = np.random.normal(miCR , 0.1)
      while (True):
        mut = scipy.stats.cauchy.rvs(loc=miF, scale=0.1)
        if (mut > 0):
          break
      if(mut > 1.0): mut =1.0
      if(crossp > 1.0): crossp =1
      if(crossp < 0): crossp = 0 

      ind = fitness.argsort()[range(best_number)] # find index of best p*popsize
      best_idx = random.choice(ind) # index of the best in p*popsize (random best)
      pbest = X[best_idx] # random best 
      idxs = [idx for idx in range(popsize) if idx != i]
      iused = np.random.choice(idxs, 1, replace = False)
      idxs = [idx for idx in range(popsize) if (idx != iused and idx != best_idx)]
      a  = X[iused]
      if(len(Xarq)>0):
        Xii = np.asarray(Xarq)
        Xnew = np.append(X[idxs,:],Xii,axis=0)
      else:
        Xnew=np.copy(X[idxs,:])

      idxs2 = [idx for idx in range(len(Xnew[:,0])) if 2 > 1]
      b  = Xnew[np.random.choice(idxs2, 1, replace = False)]

      mutant = X[i,:]+mut*(pbest-X[i,:]) + mut * (b - a)
      mutant = mutant.ravel()

      #trial = np.copy(mutant)
      trial = np.copy(X[i,:])

      jrand = random.randint(0,dim-1)

      for j in range(dim):
        rdn = np.random.rand()
        if(rdn < crossp or j == jrand) : 
          trial[j] = mutant[j] 
        else:
          trial[j] = X[i,j]

      mutant  = np.copy(trial) # just to use word mutant instead of trial
      
      for k in range(dim):
        if(mutant[k]>MAX[k]):
          mutant[k]=MAX[k]
        if(mutant[k]<MIN[k]):
          mutant[k]=MIN[k]

      Xmutant.append(mutant)

      FES = FES + 1
      fmut =fobj(mutant)
      fmutant.append(fmut)
    
    

      #for i in range(popsize):
      if( fmutant[i]  <= fitness[i]):
        Xold = X[i,:] 
        X[i,:] = Xmutant[i]
        fx.append(fitness[i])
        fu.append(fmutant[i])
        fitness[i]=fmutant[i]
        SCR.append(crossp)
        SF.append(mut)

        sizeARQ = np.array(Xarq).shape[0]
        if(sizeARQ < Narquive):
          Xarq.append(Xold)
        else:
          ri = random.randint(0, Narquive-1) # deleting individuals if necessary
          Xarq[ri] = Xold

        if(fbest >= fmutant[i]):
          best = Xmutant[i]
          fbest = fmutant[i]
      else:
        X[i,:] = X[i,:] # it is not necessary this line but just to remeber it

    
    # memory update algorithm 1:
    if(kH > H-1): kH = 0

    if (len(SCR)>0):
      deltaF = np.abs(np.asarray(fu)-np.asarray(fx))
      if( sum(deltaF) > 0):
        w = deltaF/sum(deltaF)
      else:
        w =np.ones(len(fu))
      if(MCR[kH] == terminal or max(SCR)==0):
        MCR[kH] = terminal
      else:
        y = np.asarray(SCR)
        MCR[kH] = np.dot(w,y**2)/np.dot(w,y)
      y = np.asarray(SF)
      MF[kH] = np.dot(w,y**2)/np.dot(w,y)
      kH = kH + 1

    else:
      MCR = MCR
      MF = MF

  setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]


  y=fitness


  XY= np.c_[X,y] #concatena x e y em 2 colunas            
  XYsorted = XY[XY[:,-1].argsort()] #Ordena a partir da last col(Y) for all row
  X=XYsorted[:,0:dim]
  XY=XYsorted
  fitness = XYsorted[:,-1]  
  BEST=X[0,:]
  FOBEST=fitness[0]
  BEST_XY =np.append(BEST,FOBEST)
  
  return fitness,X,BEST,FOBEST,XY,BEST_XY,FES,Xarq,setTUNE


In [ ]:

X=np.zeros((NPAR1,cols))
for i in range(NPAR1):
  for j in range(cols):
    r=np.random.random()
    X[i,j]=r*(MAX[j]-MIN[j])+MIN[j]

FES = 0
X1 = np.copy(X)
fit1 = np.asarray([fobj(ind) for ind in X1])
FES = FES + NPAR1
best_idx = np.argmin(fit1)
best = X1[best_idx]
fbest = fit1[best_idx] 
BEST1,FOBEST1 = best, fbest

In [ ]:
for i in range(H):
  MCR.append(MCRi);MF.append(MFi)
Narquive = popsize
setTUNE =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]

In [ ]:
for i in range(ITERATIONS):
  
  fit1,X1,BEST1,FOBEST1,XY,BEST_XY,FES,Xarq,setTUNE= LShade(MAX,MIN, popsize,fobj,setTUNE,best,fbest,fit1,X1,Xarq,FES)
  print('i=',i,FOBEST1)


i= 0 258.8218133168584
i= 1 258.8218133168584
i= 2 258.8218133168584
i= 3 243.49772797453772
i= 4 182.29320627763408
i= 5 57.11531356900003
i= 6 57.11531356900003
i= 7 57.11531356900003
i= 8 57.11531356900003
i= 9 56.28295367658206
i= 10 56.28295367658206
i= 11 56.28295367658206
i= 12 56.28295367658206
i= 13 56.28295367658206
i= 14 40.23334594551362
i= 15 23.53426211907911
i= 16 23.53426211907911
i= 17 23.53426211907911
i= 18 21.238071293718722
i= 19 20.91615818208073
i= 20 20.91615818208073
i= 21 16.98356114748713
i= 22 13.425352776555721
i= 23 7.822249236491007
i= 24 7.822249236491007
i= 25 7.822249236491007
i= 26 6.0617092599963485
i= 27 6.0617092599963485
i= 28 3.9344088873005454
i= 29 3.9344088873005454
i= 30 3.9344088873005454
i= 31 3.9344088873005454
i= 32 3.9344088873005454
i= 33 3.634439170447712
i= 34 3.6035876992769396
i= 35 2.5508004468700927
i= 36 2.5508004468700927
i= 37 2.5508004468700927
i= 38 1.176927224469349
i= 39 1.176927224469349
i= 40 1.176927224469349
i= 41 1.176

In [ ]:
fobj(X1[0,:])

0.05029458833680173

In [ ]:
setTUNE[2]# =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]

[0.014479019023754943,
 0.002006918074499549,
 0.00044669451391010466,
 0.009358648504472534,
 0.002328104129766799,
 0.0006333619661853769]

In [ ]:
setTUNE[3]# =[SF,SCR,MF,MCR,p, terminal, Narquive,H,kH]

[0.1, 0.1, 0.1, 0.1, 0.1, 0.1]